In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
!hostname
!pwd

dv001.ib.bridges2.psc.edu
/ocean/projects/asc170022p/mtragoza/mre-pinn/MICCAI-2023


In [2]:
import sys, os
import numpy as np
import pandas as pd

sys.path.append('..')
%aimport mre_pinn

sys.path.append('../../param_search')
%aimport param_search
ps = param_search

Using backend: pytorch



# MICCAI 2023 patient experiment

In [3]:
%pwd

'/ocean/projects/asc170022p/mtragoza/mre-pinn/MICCAI-2023'

In [26]:
template = '''\
#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --account=asc170022p
#SBATCH --partition=GPU-shared
#SBATCH --gres=gpu:1
#SBATCH -x v034
#SBATCH --time=48:00:00
#SBATCH -o %J.stdout
#SBATCH -e %J.stderr
#SBATCH --mail-type=all

hostname
pwd
module load anaconda3
conda activate /ocean/projects/asc170022p/mtragoza/mambaforge/envs/MRE-PINN

python ../../../train.py \\
    --xarray_dir {xarray_dir} \\
    --example_id {example_id} \\
    --frequency {frequency} \\
    --noise_ratio {noise_ratio} \\
    --omega {omega} \\
    --polar_input {polar_input} \\
    --savgol_filter {savgol_filter} \\
    --pde_name {pde_name} \\
    --pde_warmup_iters 10000 \\
    --pde_step_iters 5000 \\
    --pde_step_factor 10 \\
    --pde_init_weight {pde_init_weight} \\
    --pde_loss_weight {pde_loss_weight} \\
    --a_loss_weight {a_loss_weight} \\
    --anatomical {anatomical} \\
    --n_iters {n_iters} \\
    --save_prefix {job_name}    
'''
name = 'train_{data_name}_{example_id}_{omega}_{pde_name}_{a_loss_weight:.0e}'

param_space = ps.ParamSpace(
    pde_name=['hetero', 'helmholtz'], # helmholtz last
    a_loss_weight=[0, 1, 1e-2, 1e-4, 1e-1, 1e-3],
    data_name='patient',
    xarray_dir='../../../data/NAFLD/v4',
    example_id=['0006', '0020', '0024', '0029', '0043', '0047', '0126', '0135', '0155', '0164', '0172', '0173', '0210', '0213', '0219', '0222', '0234', '0235', '0291', '0344', '0378', '0384', '0395', '0401', '0415', '0457', '0461', '0491', '0492', '0496', '0509', '0510', '0516', '0525', '0564', '0604', '0612', '0615', '0628', '0639', '0648', '0650', '0653', '0655', '0659', '0693', '0704', '0717', '0727', '0734', '0737', '0740', '0747', '0748', '0898', '0900', '0929', '0931', '0932', '0940', '0954', '0964', '0965', '0979', '0989', '0995', '1028', '1033', '1039', '1045', '1075', '1076', '1077', '1083', '1103', '1106', '1108', '1110', '1119', '1121', '1123', '1134', '1144', '1149', '1205', '1207', '1215', '1217', '1230', '1261', '1271', '1296', '1303', '1311', '1329', '1337', '1341', '1343', '1344', '1382', '1395', '1400', '1404', '1417', '1426', '1427', '1433', '1435', '1448', '1453', '1456', '1464', '1671', '1679', '1694', '1699', '1706', '1712', '1714', '1715', '1720', '1722', '1727', '1736', '1748', '1785', '1786', '1790', '1791', '1793', '1795', '1798', '1806', '1807', '1819', '1829', '1839', '1843', '1851', '1853', '1896', '1899', '1903', '1917', '1935', '1940', '1948', '1967', '1979', '1980', '2001', '2007', '2029', '2034', '2046'],
    frequency=40,
    noise_ratio=0.0,
    omega=[30],
    polar_input=1,
    savgol_filter=[0],
    pde_init_weight=1e-18,
    pde_loss_weight=1e-16,
    anatomical=True,
    n_iters=100000,
)

len(param_space)

1860

In [27]:
for p in list(param_space)[:24]:
    print(name.format(**p))

train_patient_0006_30_hetero_0e+00
train_patient_0020_30_hetero_0e+00
train_patient_0024_30_hetero_0e+00
train_patient_0029_30_hetero_0e+00
train_patient_0043_30_hetero_0e+00
train_patient_0047_30_hetero_0e+00
train_patient_0126_30_hetero_0e+00
train_patient_0135_30_hetero_0e+00
train_patient_0155_30_hetero_0e+00
train_patient_0164_30_hetero_0e+00
train_patient_0172_30_hetero_0e+00
train_patient_0173_30_hetero_0e+00
train_patient_0210_30_hetero_0e+00
train_patient_0213_30_hetero_0e+00
train_patient_0219_30_hetero_0e+00
train_patient_0222_30_hetero_0e+00
train_patient_0234_30_hetero_0e+00
train_patient_0235_30_hetero_0e+00
train_patient_0291_30_hetero_0e+00
train_patient_0344_30_hetero_0e+00
train_patient_0378_30_hetero_0e+00
train_patient_0384_30_hetero_0e+00
train_patient_0395_30_hetero_0e+00
train_patient_0401_30_hetero_0e+00


In [29]:
%autoreload

expt_names = [
    '2023-2-16_patient_anat',  # a_loss_weight
    '2023-2-18_patient_anat',  # move queued jobs to GPU-shared
    '2023-2-20_patient_anat2', # expanded a_loss_weight range
    '2023-2-26_patient_FEM',   # just the new FEM baselines,
    '2023-2-28_patient_anat3', # fix v034 errors
]
expt_name = expt_names[-1]

jobs = ps.submit(template, name, list(param_space), work_dir=expt_name, verbose=True)
jobs.to_csv(f'{expt_name}.jobs')

#jobs = pd.concat([pd.read_csv(f'{e}.jobs', index_col=0) for e in expt_names])
#jobs = pd.read_csv(f'{expt_name}.jobs', index_col=0)

jobs

100%|██████████| 1860/1860 [00:13<00:00, 134.92it/s]
[14735968, 14735969, 14735970, 14735971, 14735972, 14735973, 14735974, 14735975, 14735976, 14735977, 14735978, 14735979, 14735980, 14735981, 14735982, 14735983, 14735984, 14735985, 14735986, 14735987, 14735988, 14735989, 14735990, 14735991, 14735992, 14735993, 14735994, 14735995, 14735996, 14735997, 14735998, 14735999, 14736000, 14736001, 14736002, 14736003, 14736004, 14736005, 14736006, 14736007, 14736008, 14736009, 14736010, 14736011, 14736012, 14736013, 14736014, 14736015, 14736016, 14736017, 14736018, 14736019, 14736020, 14736021, 14736022, 14736023, 14736024, 14736025, 14736026, 14736027, 14736028, 14736029, 14736030, 14736031, 14736032, 14736033, 14736034, 14736035, 14736036, 14736037, 14736038, 14736039, 14736040, 14736041, 14736042, 14736043, 14736044, 14736045, 14736046, 14736047, 14736048, 14736049, 14736050, 14736051, 14736052, 14736053, 14736054, 14736055, 14736056, 14736057, 14736058, 14736059, 14736060, 14736061, 147360

,pde_name,a_loss_weight,data_name,xarray_dir,example_id,frequency,noise_ratio,omega,polar_input,savgol_filter,...,anatomical,n_iters,job_name,job_id,partition,job_state,node_id,runtime,work_dir,array_idx
0,hetero,0.000,patient,../../../data/NAFLD/v4,0006,40,0.0,30,1,0,...,True,100000,train_patient_0006_30_hetero_0e+00,14735968,GPU-shared,PENDING,(Priority),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
1,hetero,0.000,patient,../../../data/NAFLD/v4,0020,40,0.0,30,1,0,...,True,100000,train_patient_0020_30_hetero_0e+00,14735969,GPU-shared,PENDING,(Priority),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
2,hetero,0.000,patient,../../../data/NAFLD/v4,0024,40,0.0,30,1,0,...,True,100000,train_patient_0024_30_hetero_0e+00,14735970,GPU-shared,PENDING,(Priority),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
3,hetero,0.000,patient,../../../data/NAFLD/v4,0029,40,0.0,30,1,0,...,True,100000,train_patient_0029_30_hetero_0e+00,14735971,GPU-shared,PENDING,(Priority),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
4,hetero,0.000,patient,../../../data/NAFLD/v4,0043,40,0.0,30,1,0,...,True,100000,train_patient_0043_30_hetero_0e+00,14735972,GPU-shared,PENDING,(Priority),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,helmholtz,0.001,patient,../../../data/NAFLD/v4,2001,40,0.0,30,1,0,...,True,100000,train_patient_2001_30_helmholtz_1e-03,14737823,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
1856,helmholtz,0.001,patient,../../../data/NAFLD/v4,2007,40,0.0,30,1,0,...,True,100000,train_patient_2007_30_helmholtz_1e-03,14737824,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
1857,helmholtz,0.001,patient,../../../data/NAFLD/v4,2029,40,0.0,30,1,0,...,True,100000,train_patient_2029_30_helmholtz_1e-03,14737825,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
1858,helmholtz,0.001,patient,../../../data/NAFLD/v4,2034,40,0.0,30,1,0,...,True,100000,train_patient_2034_30_helmholtz_1e-03,14737826,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN


In [42]:
%autoreload
status_cols = ['job_name', 'job_state', 'node_id', 'runtime', 'stdout', 'stderr']
status = ps.status(jobs, parse=True, output_pat=r'^([a-z0-9]+).*bridges2')
status[status_cols]

,job_name,job_state,node_id,runtime,stdout,stderr
job_id,,,,,,
14735968,train_patient_0006_30_hetero_0e+00,RUNNING,v006,5:39,v006,NaN
14735969,train_patient_0020_30_hetero_0e+00,RUNNING,v015,5:39,v015,NaN
14735970,train_patient_0024_30_hetero_0e+00,RUNNING,v023,5:39,v023,NaN
14735971,train_patient_0029_30_hetero_0e+00,RUNNING,v021,5:39,v021,NaN
14735972,train_patient_0043_30_hetero_0e+00,RUNNING,v021,5:39,v021,NaN
...,...,...,...,...,...,...
14737823,train_patient_2001_30_helmholtz_1e-03,PENDING,(Priority),0:00,NaN,NaN
14737824,train_patient_2007_30_helmholtz_1e-03,PENDING,(Priority),0:00,NaN,NaN
14737825,train_patient_2029_30_helmholtz_1e-03,PENDING,(Priority),0:00,NaN,NaN


In [43]:
def get_error_type(e):
    for error_type in [
        'CANCELLED',
        'python: command not found',
        'Unexpected error from cudaGetDeviceCount()'
    ]:
        if error_type in e:
            return error_type
    return e

status['has_stderr_file'] = status.stderr.map(lambda x: np.isfinite(x))
status['has_stderr'] = ~status.stderr.isnull()
status['error'] = status.stderr.astype(str).map(get_error_type)

status.fillna('DONE').groupby(['job_state', 'has_stderr_file', 'has_stderr', 'error'])[['job_name']].count()

,,,,job_name
job_state,has_stderr_file,has_stderr,error,
PENDING,False,False,nan,1839
RUNNING,False,False,nan,21


In [45]:
status[status.has_stderr].groupby(['error', 'stdout'])[['job_name']].count()

,,job_name
error,stdout,


In [ ]:
pinn_jobs = pd.concat([
    pd.read_csv(f'2023-2-16_patient_anat.jobs', index_col=0),
    pd.read_csv(f'2023-2-18_patient_anat.jobs', index_col=0),
    pd.read_csv(f'2023-2-20_patient_anat2.jobs', index_col=0)
])
fem_jobs = pd.read_csv(f'2023-2-26_patient_FEM.jobs', index_col=0)

param_cols = ['pde_name', 'example_id', 'omega', 'a_loss_weight', 'savgol_filter'] # experimental parameters
index_cols = ['variable_name', 'spatial_frequency_bin', 'spatial_region'] # metric identifiers
metric_cols = ['MSAV', 'PSD', 'MAV', 'R'] # metric values

pinn_metrics = ps.metrics(pinn_jobs)
pinn_metrics['savgol_filter'] = 0

pinn_metrics_sg = pinn_metrics.copy()
pinn_metrics_sg['savgol_filter'] = 1

pinn_metrics = pd.concat([pinn_metrics, pinn_metrics_sg])

fem_metrics = ps.metrics(fem_jobs)
fem_metrics.iteration *= 100000

In [ ]:
param_cols = ['pde_name', 'example_id', 'omega', 'a_loss_weight', 'savgol_filter'] # experimental parameters
index_cols = ['variable_name', 'spatial_frequency_bin', 'spatial_region'] # metric identifiers
metric_cols = ['MSAV', 'PSD', 'MAV', 'R'] # metric values

def metric_map(tup):
    metric_name, var_name = tup
    new_col_name = f'{var_name}_{metric_name}'
    new_col_name = new_col_name.replace('diff_MSAV', 'pred_MSAE')
    new_col_name = new_col_name.replace('pde_diff_MSAV', 'PDE_MSAE')
    new_col_name = new_col_name.replace('diff_MAV', 'pred_MAD')
    return new_col_name

def format_metrics(metrics):
    metrics = metrics[metrics.iteration == 100e3]
    metrics = metrics.groupby(param_cols + index_cols, sort=False)[metric_cols].mean()
    metrics = metrics.unstack(level=[len(param_cols)])
    metrics.columns = [metric_map(t) for t in metrics.columns.to_flat_index()]
    return metrics

def replace_metrics(m1, m2, index1, index2, columns):
    m1 = m1.reset_index().set_index(index2)
    m2 = m2.reset_index().set_index(index2)
    m1[columns] = m2[columns]
    return m1.reset_index().set_index(index1)
    
pinn_metrics = format_metrics(pinn_metrics)
fem_metrics  = format_metrics(fem_metrics)

metrics = replace_metrics(
    pinn_metrics, fem_metrics,
    index1=param_cols + index_cols[1:],
    index2=['pde_name', 'example_id', 'savgol_filter'] + index_cols[1:],
    columns=[c for c in fem_metrics if c.startswith('fem')]
)
metrics

In [ ]:
m = pinn_metrics.reset_index()

fig = ps.plot(
    m[m.spatial_region == 'all'],
    x=['a_loss_weight'],
    y=['u_pred_MSAE', 'u_pred_R'],
    height=2.5,
    width=2.5,
    legend=False,
    tight=True
)

In [ ]:
fig = ps.plot(
    m[m.spatial_region == 'all'],
    x=['a_loss_weight'],
    y=['mu_pred_MSAE', 'mu_pred_R'],
    height=2.5,
    width=2.5,
    legend=False,
    tight=True
)

In [ ]:
m['method'] = 'PINN_' + m['pde_name']

direct_m = m[m.pde_name == 'hetero'].copy()
direct_m['method'] = 'direct_helmholtz'
for col in m.columns:
    if col.startswith('direct'):
        direct_m[col.replace('direct', 'mu')] = m[col]
        
hh_fem_m = m[m.pde_name == 'helmholtz'].copy()
hh_fem_m['method'] = 'FEM_helmholtz'
for col in m.columns:
    if col.startswith('fem'):
        hh_fem_m[col.replace('fem', 'mu')] = m[col]

ht_fem_m = m[m.pde_name == 'hetero'].copy()
ht_fem_m['method'] = 'FEM_hetero'
for col in m.columns:
    if col.startswith('fem'):
        ht_fem_m[col.replace('fem', 'mu')] = m[col]
        
mm = pd.concat([direct_m, ht_fem_m, m])

method_order = ['direct_helmholtz', 'FEM_helmholtz', 'FEM_hetero', 'PINN_helmholtz', 'PINN_hetero']
mm['method_index'] = mm['method'].map(method_order.index)
mm = mm.sort_values('method_index')

In [ ]:
import matplotlib as mpl
pct_format = mpl.ticker.PercentFormatter()

mm['mu_pred_MSAE_relative'] = mm['mu_pred_MSAE'] / mm['mu_true_MSAV'] * 100
mm['mu_pred_MAD_relative'] = mm['mu_pred_MAD'] / mm['mu_true_MAV'] * 100

In [ ]:
%autoreload
import seaborn as sns
import matplotlib.pyplot as plt

colors = ps.results.get_color_palette(10, type='deep', min_val=0)
blue, orange, green, red, purple, brown, pink, gray, yellow, cyan = colors

colors = [blue, yellow, red, cyan, green]
#colors = [blue, yellow, cyan]
sns.set_palette(colors)
colors = sns.color_palette()
colors

In [ ]:
mm.spatial_region.unique()

In [ ]:
fig = ps.plot(
    mm[~mm.spatial_region.isin({'0.0', 'all'}) & ~mm.savgol_filter],
    x='method',
    y='mu_pred_MAD_relative',
    height=3,
    width=7,
    legend=True,
    tight=True,
    plot_func=ps.results.barplot
)
fig.axes[0].set_ylabel('$\mu$ relative MAD (%)')
fig.axes[0].set_ylim(0, 100)
fig.axes[0].yaxis.set_major_formatter(pct_format)
fig.tight_layout()

for i, patch in enumerate(fig.axes[0].patches):
    plt.setp(patch, facecolor=colors[i])   

for ext in ['png', 'pdf']:
    fig.savefig(f'images/patient_method_bar_plot.{ext}', bbox_inches='tight', dpi=200)

In [ ]:
fig = ps.plot(
    mm[(mm.spatial_region == 'all') & (mm.spatial_frequency_bin == 'all') & ~mm.savgol_filter].copy(), # across all omegas
    x='method',
    hue='method',
    y='mu_pred_R',
    height=3,
    width=7,
    legend=False,
    tight=True,
    plot_func=ps.results.barplot
)
fig.axes[0].set_ylabel('$\mu$ correlation (R)')
fig.axes[0].set_ylim([0, 1])

for i, patch in enumerate(fig.axes[0].patches):
    plt.setp(patch, facecolor=colors[i])

for ext in ['png', 'pdf']:
    fig.savefig(f'images/patient_method_R_bar_plot.{ext}', bbox_inches='tight', dpi=200)

In [ ]:
fig = ps.plot(
    mm[~mm.spatial_region.isin({'0', 'all'}) & mm.method.str.startswith('PINN')], 
    x='omega',
    y='mu_pred_MAD_relative',
    height=5,
    width=6,
    legend=True,
    tight=True,
    plot_func=ps.results.barplot
)
fig.axes[0].set_ylabel('$\mu$ relative MAD (%)')
fig.axes[0].yaxis.set_major_formatter(pct_format)
fig.axes[0].set_ylim(0, 100)
fig.tight_layout()

fig.axes[0].set_xlabel('omega')

for ext in ['png', 'pdf']:
    fig.savefig(f'images/patient_omega_bar_plot.{ext}', bbox_inches='tight', dpi=200)

In [ ]:
mm['a_loss_weight'][mm.a_loss_weight == 0] = 1e-6

In [ ]:
mm['mu_pred_R2'] = mm['mu_pred_R']**2

fig = ps.plot(
    mm[mm.spatial_region.isin({'all'}) & mm.method.str.startswith('PINN')], 
    x='a_loss_weight',
    y='mu_pred_R2',
    height=5,
    width=6,
    legend=True,
    tight=True,
    plot_func=sns.lineplot
)
fig.axes[0].set_ylabel('$\mu$ correlation (R)')
fig.axes[0].set_xlabel('anatomic loss weight')
fig.axes[0].set_xscale('log')
fig.axes[0].set_xticks([1e-6, 1e-4, 1e-2, 1e0, 1e2])
fig.axes[0].set_xticklabels(['0', '1e-4', '1e-2', '1e0', '1e2'])
fig.tight_layout()

for ext in ['png', 'pdf']:
    fig.savefig(f'images/patient_anat_bar_plot.{ext}', bbox_inches='tight', dpi=200)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8.5, 3), width_ratios=(0.7, 0.3), squeeze=False)

sns.boxplot(
    mm[mm.spatial_region.isin({'all'}) & mm.savgol_filter & mm.a_loss_weight == 0],
    x='method',
    y='mu_pred_R',
    ax=ax[0,0]
)
ax[0,0].grid(linestyle=':')
ax[0,0].axes.set_axisbelow(True)
#ax[0,0].legend(frameon=True, fontsize='x-small', loc='lower left', ncol=3, edgecolor='w')

t = mpl.transforms.ScaledTranslation(-0.6, 0, fig.dpi_scale_trans)
ax[0,0].text(
    0, 1, 'A', fontweight='bold', va='bottom', ha='right',
    transform=ax[0,0].transAxes + t
)

ax[0,0].set_ylabel('$\mu$ correlation (R)')
ax[0,0].set_ylim(0, 1.0)
#ax[0,0].yaxis.set_major_formatter(pct_format)

ax[0,0].set_xlabel('method')
#ax[0,0].set_xticklabels(['background', 'target 1', 'target 2', 'target 3', 'target 4'])

sns.lineplot(
    mm[mm.spatial_region.isin({'all'}) & mm.method.str.startswith('PINN')],
    x='a_loss_weight',
    y='mu_pred_MSAE_relative',
    ax=ax[0,1]
)
ax[0,1].grid(linestyle=':')
ax[0,1].axes.set_axisbelow(True)

t = mpl.transforms.ScaledTranslation(-0.4, 0, fig.dpi_scale_trans)
ax[0,1].text(0, 1, 'B', fontweight='bold', va='bottom', ha='right', transform=ax[0,1].transAxes + t)

ax[0,1].set_ylabel('$\mu$ relative MSAE (%)')
ax[0,1].set_ylim(0, 50)
#ax[0,1].yaxis.set_major_formatter(pct_format)

ax[0,1].set_xlabel('anatomic loss weight')
ax[0,1].set_xscale('log')
ax[0,1].set_xticks([1e-6, 1e-4, 1e-2, 1e0, 1e2])
ax[0,1].set_xticklabels(['0'] + ax[0,1].get_xticklabels()[1:])

sns.despine(fig)
fig.tight_layout()

for ext in ['png', 'pdf']:
    fig.savefig(f'images/patient_plots.{ext}', bbox_inches='tight', dpi=200)

In [ ]:
%autoreload

blue, orange, green, red, purple, brown, pink, gray, yellow, cyan = sns.color_palette('tab10')

mre_pinn.visual.COLORS.update(
    red=red,
    yellow=yellow,
    green=green,
    cyan=cyan,
    blue=blue
)

In [ ]:
example = mre_pinn.data.MREExample.load_xarrays('../data/NAFLD/v3', '0006', anat=True)
example.arrays['anat'] = example.anat.sel(z=0.018237183332, method='nearest')
example.view('anat', ax_height=2, space=0.2)
plt.gcf().tight_layout()
plt.savefig('images/patient_anat.png', dpi=200, bbox_inches='tight')
example.view('wave', ax_height=2, space=0.2)
plt.gcf().tight_layout()
plt.savefig('images/patient_wave.png', dpi=200, bbox_inches='tight')
example.view('mre', ax_height=2, space=0.2)
plt.gcf().tight_layout()
plt.savefig('images/patient_mre.png', dpi=200, bbox_inches='tight')

In [ ]:
%autoreload
import mre_pinn

image_names = [
    'wave image', 'ground truth', 'direct_helmholtz', 'FEM_hetero', 'PINN_hetero', 'PINN_hetero_anat'
]

pinn_expt_name = '2023-2-18_patient_anat'
pinn_job_name = 'train_patient_{example_id}_{omega}_{pde_name}_{a_loss_weight:.0e}'

fem_expt_name = '2023-2-26_patient_FEM'
fem_job_name = 'train_patient_{example_id}_{savgol_filter}_{pde_name}'

def plot_image_grid(example_ids):
    n_rows = len(example_ids)
    n_cols = len(image_names)
    ax_width = 1.2
    ax_height = ax_width
    
    omega = 30
    a_loss_weight = 1
    savgol_filter = 0

    fig, axes, cbar_ax = mre_pinn.visual.subplot_grid(
        n_rows, n_cols, ax_height, ax_width, space=0, pad=(0.35,0.15,0.15,0.25)
    )
    for row_idx, example_id in enumerate(example_ids):
        example = mre_pinn.data.MREExample.load_xarrays('../data/NAFLD/v4', example_id)
        z = example.mre.z[0]
        for col_idx, image_name in enumerate(image_names):
            ax = axes[row_idx,col_idx]
            #ax.text(0.1, 0.1, f'{row_idx}, {col_idx}')
            if col_idx == 0:
                ax.set_ylabel(example_id, fontsize='medium')
            if row_idx == 0:
                ax.set_title(image_name, fontsize='small')
            ax.set_yticks([])
            ax.set_xticks([])

            if image_name == 'wave image':
                array = example.wave[...,0]
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = array.real * (example.mre_mask.values[...,0] > 0)

            elif image_name == 'ground truth':
                array = example.mre[...,0]
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array) * (example.mre_mask.values[...,0] > 0)

            elif image_name == 'direct_helmholtz':
                job_name = fem_job_name.format(
                    example_id=example_id, pde_name='helmholtz', savgol_filter=savgol_filter
                )
                nc_file = f'{fem_expt_name}/{job_name}/{job_name}_direct.nc'
                array = mre_pinn.data.dataset.load_xarray_file(nc_file).sel(variable='direct_pred', z=z)
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array)
                
            elif image_name == 'FEM_helmholtz':
                job_name = fem_job_name.format(
                    example_id=example_id, pde_name='helmholtz', savgol_filter=savgol_filter
                )
                nc_file = f'{fem_expt_name}/{job_name}/{job_name}_fem.nc'
                array = mre_pinn.data.dataset.load_xarray_file(nc_file).sel(variable='fem_pred', z=z)
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array)

            elif image_name == 'FEM_hetero':
                job_name = fem_job_name.format(
                    example_id=example_id, pde_name='hetero', savgol_filter=savgol_filter
                )
                nc_file = f'{fem_expt_name}/{job_name}/{job_name}_fem.nc'
                array = mre_pinn.data.dataset.load_xarray_file(nc_file).sel(variable='fem_pred', z=z)
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array)

            elif image_name == 'PINN_helmholtz':
                job_name = pinn_job_name.format(
                    example_id=example_id, pde_name='helmholtz', omega=omega, a_loss_weight=a_loss_weight
                )
                nc_file = f'{pinn_expt_name}/{job_name}/{job_name}_elastogram.nc'
                array = mre_pinn.data.dataset.load_xarray_file(nc_file).sel(variable='mu_pred', z=z)
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array)

            elif image_name == 'PINN_hetero':
                job_name = pinn_job_name.format(
                    example_id=example_id, pde_name='hetero', omega=omega, a_loss_weight=0
                )
                nc_file = f'{pinn_expt_name}/{job_name}/{job_name}_elastogram.nc'
                array = mre_pinn.data.dataset.load_xarray_file(nc_file).sel(variable='mu_pred', z=z)
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array)
                
            elif image_name == 'PINN_hetero_anat':
                job_name = pinn_job_name.format(
                    example_id=example_id, pde_name='hetero', omega=omega, a_loss_weight=a_loss_weight
                )
                nc_file = f'{pinn_expt_name}/{job_name}/{job_name}_elastogram.nc'
                array = mre_pinn.data.dataset.load_xarray_file(nc_file).sel(variable='mu_pred', z=z)
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array)

            mre_pinn.visual.imshow(ax, array[:-24,8:-16], **color_kws)
    
    return fig

#fig0 = plot_image_grid(['0006', '0020', '0024', '0029'])
#fig1 = plot_image_grid(['0043', '0047', '0126', '0135'])
#fig = plot_image_grid(['2029', '1948', '1940', '1839', '1791', '1671', '1448', '1435', '1045', '1039'])
fig = plot_image_grid(['1791', '1039', '1948'])

for ext in ['png', 'pdf']:
    fig.savefig(f'images/patient_image_grid.{ext}', bbox_inches='tight', dpi=200)
    pass

In [ ]:
example = mre_pinn.data.MREExample.load_xarrays('../data/NAFLD/v3', '0006')
example.view()

In [ ]:
import xarray as xr
u = example.wave
x = u.field.points(reshape=False)
x = (x - x.mean(axis=(0,1,2), keepdims=True)) / (x.max(axis=(0,1,2), keepdims=True) - x.min(axis=(0,1,2), keepdims=True))
x, y, z = x[...,0], x[...,1], x[...,2]
r = np.sqrt(x**2 + y**2)
sin = y / (r + 1e-3)
cos = x / (r + 1e-3)
x = np.stack([x, y, z, r, sin, cos], axis=-1)

print(x.shape)
x_dims = ['x', 'y', 'z', 'component']
x_coords = {
    'x': u.x,
    'y': u.y,
    'z': u.z,
    'component': ['x', 'y', 'z', 'r', 'sin', 'cos']
}
x = xr.DataArray(x, coords=x_coords, dims=x_dims)
x.name = 'position'
mre_pinn.visual.XArrayViewer(x, ax_height=3)

In [ ]:
example.wave.coords